In [1]:
!nvidia-smi

Thu Mar 31 12:56:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.56       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:01:00.0 Off |                  N/A |
| 30%   47C    P8    21W / 300W |      0MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U 'spacy[cuda112]'
!python -m spacy download en_core_web_trf

/opt/conda/lib/python3.8/site-packages/cupy/_environment.py:436: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda112, cupy-cuda115

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
     |████████████████████████████████| 460.2 MB 97 kB/s s eta 0:00:013
     |████████████████████████████████| 51 kB 153 kB/

In [6]:
import spacy
from tqdm.auto import tqdm
tqdm.pandas()
import pandas as pd

In [7]:
spacy.require_gpu()

True

In [8]:
nlp = spacy.load("en_core_web_trf") # spacy model

In [40]:
def get_orgs(text):
    '''
    This function takes a text. Uses the Spacy model.
    The model will tokenize, POS-tag and recognize the entities named in the text.
    Returns a list of entities in the text that were recognized as organizations.
    '''
    # Apply the model
    tags = nlp(text)
    ents = [(ent.text.replace('\'', ''), (ent.start_char, ent.end_char)) for ent in tags.ents if ent.label_=='ORG'] # also remove apostrophes
    # Return the list of entities recognized as organizations
    return ents

# test run
get_orgs('Apple and Microsoft plan to form a joint venture for the development of cloud-based '
             'computing infrastrucutre.')

[('Apple', (0, 5)), ('Microsoft', (10, 19))]

In [9]:
# read lexis nexis news data
df = pd.read_pickle("https://www.dropbox.com/s/36a07va701ap6h0/lexisnexis_firm_alliances_2017_cleaned_min_2_companies.pkl?dl=1")
df

,title,content,publication,word_count,publication_date,publication_date_text,author,copyright,subject,country,city,person,industry,company,lang
113,"""Anhui Jianghuai Automobile Group Corp., Ltd.""...","(600418) ""Anhui Jianghuai Automobile Group Cor...",['Shanghai Stock Exchange'],459,2017-11-28,"November 28, 2017 Tuesday",NaN,Copyright 2017 Shanghai Stock Exchange All Rig...,"[ALLIANCES & PARTNERSHIPS, STOCK EXCHANGES]","[CHINA, ASIA]","[SHANGHAI, CHINA]",[],[SIC3711 MOTOR VEHICLES & PASSENGER CAR BODIES...,"[ANHUI JIANGHUAI AUTOMOBILE CO LTD, VOLKSWAGEN...",en
659,"""Forced Labour"" Case To Go To Trial","Dec 08, 2017( Mondaq: http://www.mondaq.com / ...",['Mondaq'],7046,2017-12-08,"December 8, 2017 Friday","Mr Rick Williams, Tim Pritchard and Auke Visser",Copyright 2017 Newstex LLC All Rights Reserved,"[APPEALS, APPEALS COURTS, APPELLATE DECISIONS,...","[ERITREA, CANADA]","[ASMARA, ERITREA]",[],"[NAICS212221 GOLD ORE MINING, SIC1041 GOLD ORE...","[NEVSUN RESOURCES LTD, TAHOE RESOURCES INC]",en
699,"""GD Power Development Co., Ltd."" (the Company)...",The Company held the 48th session of the 7th d...,['Shanghai Stock Exchange'],803,2017-08-29,"August 29, 2017 Tuesday",NaN,Copyright 2017 Shanghai Stock Exchange All Rig...,"[JOINT VENTURES, STOCK EXCHANGES, AGREEMENTS]","[CHINA, ASIA]","[SHANGHAI, CHINA, BEIJING, CHINA]",[],"[NAICS221122 ELECTRIC POWER DISTRIBUTION, NAIC...","[GD POWER DEVELOPMENT CO LTD, CHINA SHENHUA EN...",en
1667,"""Shanghai Chlor-Alkali Chemical Co., Ltd."" (th...","(600618) ""Shanghai Chlor-Alkali Chemical Co., ...",['Shanghai Stock Exchange'],425,2017-01-12,"January 12, 2017 Thursday",NaN,Copyright 2017 Shanghai Stock Exchange All Rig...,[STOCK EXCHANGES],"[CHINA, ASIA]","[SHANGHAI, CHINA]",[],[NAICS325180 OTHER BASIC INORGANIC CHEMICAL MA...,"[SHANGHAI CHLOR-ALKALI CHEMICAL CO, ALLANFIELD...",en
1821,"""Sinoma Energy Conservation Ltd."" (the Company...",The Company held the 22nd session of the 2nd d...,['Shanghai Stock Exchange'],1008,2017-07-11,"July 11, 2017 Tuesday",NaN,Copyright 2017 Shanghai Stock Exchange All Rig...,"[ENERGY DEVELOPMENT PROGRAMS, ENGINEERING, HOL...","[CHINA, PHILIPPINES, ASIA, EGYPT]","[SHANGHAI, CHINA, WUHAN, HUBEI, CHINA]",[],"[SIC8711 ENGINEERING SERVICES, SIC3241 CEMENT,...","[APO CEMENT CORP, CHINA NATIONAL MATERIALS CO ...",en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492128,£30m centre aiming to foster the next Facebook...,A£30M research centre that aims to ensure the ...,"['The Journal (Newcastle, UK)']",3064,2017-09-19,"September 19, 2017 Tuesday",COREENA FORD,Copyright 2017 Newcastle Chronicle & Journal L...,"[INTERNET SOCIAL NETWORKING, ARCHITECTURE, ASS...","[UNITED KINGDOM, ENGLAND]","[NEWCASTLE UPON TYNE, ENGLAND]",[],[NAICS519130 INTERNET PUBLISHING & BROADCASTIN...,"[FACEBOOK INC, GOOGLE INC]",en
1492515,Â BreadTalk plans 200 stores in 7 years,"BTM (Thailand) Ltd, a joint venture between Si...",['The Nation (Thailand)'],3332,2017-06-01,"June 1, 2017 Thursday",NaN,"Copyright 2017 Nation News Network Co., Ltd. A...","[BUSINESS OPERATIONS, JOINT VENTURES, MANAGERS...","[THAILAND, SINGAPORE, MYANMAR, MALAYSIA, BRUNE...","[BANGKOK, THAILAND]",[],[NAICS531110 LESSORS OF RESIDENTIAL BUILDINGS ...,"[MINOR FOOD GROUP PCL, AL MUDON INTERNATIONAL ...",en
1492517,Â Sansiri partners with Tokyu on Bt2 bn Ekkam...,LISTED property firm Sansiri Plc has set up a ...,['The Nation (Thailand)'],3960,2017-08-09,"August 9, 2017 Wednesday",NaN,"Copyright 2017 Nation News Network Co., Ltd. A...","[HOLDING COMPANIES, JOINT VENTURES, MANAGERS &...","[JAPAN, THAILAND]","[BANGKOK, THAILAND, TOKYO, JAPAN]",[],"[NAICS485112 COMMUTER RAIL SYSTEMS, NAICS23622...","[TOKYU CORP, SANSIRI PCL, TOKYU LIVABLE INC, B...",en
1492884,Čibuk 1 Wind Farm To Receive EUR215.4mn Loan,News: The European Bank for Reconstruction and...,['Business Monitor Online'],1186,2017-10-20,"October 20, 2017 Friday",NaN,Copyright 2017 Business Monitor International ...,"[INTERNATIONAL ECONOMIC ORGANIZA

In [41]:
df['ents_pred'] = df.content.str[:600].progress_apply(get_orgs)

  0%|          | 0/24825 [00:00<?, ?it/s]

In [58]:
df.ents_pred.sample().iloc[0]

[('RTTNews', (1, 8)),
 ('Qualcomm Inc.', (12, 25)),
 ('QCOM', (27, 31)),
 ('TDK Corp.', (49, 58)),
 ('TTDKF.PK', (60, 68)),
 ('TTDKY.PK', (70, 78)),
 ('RF360 Holdings Singapore PTE. Ltd.', (159, 193)),
 ('Qualcomm', (224, 232)),
 ('RFFE Business Unit', (235, 253)),
 ('TDK SAW Business Group', (535, 557)),
 ('RF360 Holdings', (584, 598))]

In [46]:
df['firms_pred'] = df.ents_pred.apply(lambda orgs: [org[0] for org in orgs])
df['spans_pred'] = df.ents_pred.apply(lambda orgs: [org[1] for org in orgs])

In [59]:
df.to_pickle('lexis_2017_with_org_preds_spacy.pkl')